In [1]:
import pandas as pd
import requests
from datetime import date, timedelta
import json
from pymongo import MongoClient 


In [2]:
client = MongoClient() 
client = MongoClient("mongodb://localhost:27017")

In [3]:
mydatabase = client['Zones'] 
mycollection = mydatabase['NiftySmCap250H'] 

In [4]:
def fiblvlprice(lvl,ind):
    return (explosive['Low'][ind]-explosive['High'][ind])*lvl+explosive['High'][ind]
def fiblvlpricebase(lvl,ind):
    return (df['low'][ind]-df['high'][ind])*lvl+df['high'][ind]


In [5]:
def wwma(values, n):
    """
     J. Welles Wilder's EMA 
    """
    return values.ewm(alpha=1/n, adjust=False).mean()

def atr(df, n=14):
    data = df.copy()
    high = data['high']
    low = data['low']
    close = data['close']
    data['tr0'] = abs(high - low)
    data['tr1'] = abs(high - close.shift())
    data['tr2'] = abs(low - close.shift())
    tr = data[['tr0', 'tr1', 'tr2']].max(axis=1)
    atr = wwma(tr, n)
    return atr

In [6]:
def isCandleExplosive(high,low,openn,close,atravg):
    mp = 1.2
    ratio = 50
    cand = 20
    rang = atravg

    candr = abs(high-low)
    bodyr = abs(openn-close)
    if (bodyr/candr)>=ratio/100 and candr>=(rang*mp):
        return True
    else:
        return False

def get_mondays_fridays(a, b):

    start_date = pd.to_datetime(a)
    end_date = pd.to_datetime(b)

  # Ensure start_date is earlier or equal to end_date
    if start_date > end_date:
        start_date, end_date = end_date, start_date

  # Create a date range (optimized for efficiency)
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')

  # Filter for Mondays and Fridays using boolean indexing
    mondays = date_range[date_range.weekday == 0]
    fridays = date_range[date_range.weekday == 4]

    return mondays, fridays

In [7]:
def checkdata(df):
    ind = []
    date = []
    o = []
    h = []
    l = []
    c = []
    explosive = pd.DataFrame()

    for i in range(0,len(df['date'])-1):
        if isCandleExplosive(df['high'][i],df['low'][i],df['open'][i],df['close'][i],df['atravg'][i]):
            #print(df['date'][i],df['atr'][i])
            ind.append(i)
            date.append(df['date'][i])
            o.append(df['open'][i])
            h.append(df['high'][i])
            l.append(df['low'][i])
            c.append(df['close'][i])
    explosive['index'] = ind
    explosive['Date'] = date
    explosive['Open'] = o
    explosive['High'] = h
    explosive['Low'] = l
    explosive['Close'] = c
    return explosive

In [8]:

def fetchData(id):
    #url = "https://kite.zerodha.com/oms/instruments/historical/"+id+"/day?user_id=EM9701&oi=1&from="+frm+"&to="+to+""
    url = "https://kite.zerodha.com/oms/instruments/historical/"+id+"/60minute?user_id=EM9701&oi=1&from=2023-12-02&to=2024-03-01"
    headers = {
        "accept": "*/*",
        "accept-language": "en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7",
        "authorization": "enctoken iuq9znvQpSpSCOCgz/4PKeUGJoeEq0RbpNVgaZsvfv0qYyBfl1QdYtFYuJM//5bWf7TFpMmmvf7ETXl8MYhi+q80ccmOi9yxISmhuQ+s5dnotTXFOwML5g==",  # Replace with your actual authorization token
        "sec-ch-ua": "\"Not A(Brand\";v=\"99\", \"Google Chrome\";v=\"121\", \"Chromium\";v=\"121\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\"",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin"
    }

    response = requests.get(url, headers=headers)

    # Check for successful response
    if response.status_code == 200:
        #data = response.json()  # Assuming response is JSON
        # Process the data here
        #print(data)
        return response.json()
    else:
        print(f"Error: {response.status_code}")

    

In [9]:
def createDffromData(data):
    column_names = ['date', 'open', 'high', 'low', 'close', 'volume', 'oi']
    df = pd.DataFrame(data['data']['candles'], columns=column_names)
    for i in range(0,len(df['open'])):
        df['date'][i]=df['date'][i].split('T')[0]
    return df

In [10]:
def createDffromDataS(data):
    column_names = ['date', 'open', 'high', 'low', 'close', 'volume', 'oi']
    df = pd.DataFrame(data['data']['candles'], columns=column_names)
    return df

In [11]:
def putAtrValues(df):
    atri = atr(df,1)
    atr_avg = atr(df,20)
    df = df.merge(atr_avg.rename('atravg'), left_index=True, right_index=True)
    df = df.merge(atri.rename('atr'),left_index=True, right_index=True)
    return df

In [12]:
def saveDataframes(collection):
    for i in range(0,len(tiks)):
        a = createDffromDataS(data[i][tiks[i]])
        c = a.to_json()
        print(collection.insert_one({'ticker':b[i],'dataframe':str(c)}))
    

In [13]:
def jsonToDf(ticker,collection):
    return pd.read_json((collection.find_one({'ticker':ticker}))['dataframe'])

In [14]:

def greenExplosive(explosive,df):
#Green explosive
    obj = []
    for i in range(0,len(explosive['Date'])-1):
        base = explosive['index'][i]-1
        if explosive['Close'][i] > explosive['Open'][i] and abs(df['close'][base]-df['open'][base]) <= (explosive['High'][i]-explosive['Low'][i])*0.2 and explosive['Close'][i] > fiblvlprice(0.4,i):
            obj.append({"l":1,"legoutDate":explosive['Date'][i],"baseHigh":df['high'][base],"baseLow":df['low'][base],"dist":((df['high'][base]-ltp)/ltp)*100})
            #epm and open[1] > open and close[1] < low and open[1] < fibprice(0.75) and high-low > high[1]-low[1]
        elif explosive['Close'][i] > explosive['Open'][i] and df['open'][base] > explosive['Open'][i] and df['close'][base] < explosive['Low'][i] and df['open'][base] < fiblvlprice(0.75,i) and explosive['High'][i]-explosive['Low'][i] > df['high'][base]-df['low'][base]:
            obj.append({"l":2,"legoutDate":explosive['Date'][i],"baseHigh":df['high'][base],"baseLow":df['low'][base],"dist":((df['high'][base]-ltp)/ltp)*100})
            #close > open and epm and close[1] < fibprice(0.95) and open[1] > fibprice(1.25) and close[1] > open[1] and close > fibprice(0.2)
        elif explosive['Close'][i] > explosive['Open'][i] and df['close'][base] < fiblvlprice(0.95,i) and df['open'][base] > fiblvlprice(1.25,i) and df['close'][base] > df['open'][base] and explosive['Close'][i] > fiblvlprice(0.4,i):
            obj.append({"l":3,"legoutDate":explosive['Date'][i],"baseHigh":df['high'][base],"baseLow":df['low'][base],"dist":((df['high'][base]-ltp)/ltp)*100})
            #epm and checkfullbar(0) and checkfullbar(1) and high[2]-low[2] < high[1]-low[1] and checkbarexp(1) and close > open and open[2] < fibpriceind(0.5,1) and close[2] < fibpriceind(0.5,1)
    return obj

In [15]:
def find_nearest_dist_to_zero(data):
    results = []
    for token_item in data:
        nearest_dist = float('inf')
        nearest_zone = None

        for zone_item in token_item['zone']:
            dist = zone_item['dist']

            # Skip non-negative 'dist' values to focus on negative ones
            if dist >= 0:
                continue

            # Calculate absolute difference for nearest to zero comparison
            abs_diff = abs(dist)

            # Update nearest_dist and nearest_zone if closer to zero
            if abs_diff < nearest_dist:
                nearest_dist = abs_diff
                nearest_zone = zone_item

        results.append({
            'token': token_item['token'],
            'nearest_dist': nearest_dist,
            'nearest_zone': nearest_zone
        })

    return results


# Main Code

In [16]:
def convert_to_dict(data):
    result = {}
    for item in data:
        key, value = item.split("/")
        result[key] = value
    return result

# Example usage
data =["ABB/3329", "ACC/5633", "ADANIGREEN/912129", "AARTIIND/1793", "AMBUJACEM/325121",
       "ATGL/1552897", "AWL/2076161", "BAJAJHLDNG/78081", "BANKBARODA/1195009", "BEL/98049",
       "BERGEPAINT/103425", "CANBK/2763265", "CHOLAFIN/175361", "COLPAL/3876097", "DABUR/197633",
       "DLF/3771393", "DMART/5097729", "GAIL/1207553", "GODREJCP/2585345", "HAL/589569",
       "HAVELLS/2513665", "HDFCAMC/1086465", "ICICIGI/5573121", "ICICIPRULI/4774913", "INDIGO/2865921",
       "INDUSTOWER/7458561", "IOC/415745", "IRCTC/3484417", "JINDALSTEL/1723649", "LICI/2426881",
       "MARICO/1041153", "MCDOWELL-N/2674433", "MOTHERSON/1076225", "MUTHOOTFIN/6054401", "NAUKRI/3520257",
       "NYKAA/1675521", "PIDILITIND/681985", "PIIND/6191105", "SBICARD/4600577", "SIEMENS/806401",
       "SRF/837889", "TATAPOWER/877057", "TORNTPHARM/900609", "VBL/4843777", "VEDL/784129", "ZOMATO/1304833"]


# your full data here
dictionary = convert_to_dict(data)

a = {"NiftyNxt50":dictionary}
data = [
    "ABBOTINDIA/4583169",
    "ABCAPITAL/5533185",
    "ALKEM/2995969",
    "ASHOKLEY/54273",
    "ASTRAL/3691009",
    "AUBANK/5436929",
    "AUROPHARMA/70401",
    "BALKRISIND/85761",
    "BANDHANBNK/579329",
    "BATAINDIA/94977",
    "BHARATFORG/108033",
    "BIOCON/2911489",
    "COFORGE/2955009",
    "CONCOR/1215745",
    "CUMMINSIND/486657",
    "ESCORTS/245249",
    "FEDERALBNK/261889",
    "GODREJPROP/4576001",
    "GUJGASLTD/2713345",
    "HINDPETRO/359937",
    "OBEROIRLTY/5181953",
    "OFSS/2748929",
    "PERSISTENT/4701441",
    "PETRONET/2905857",
    "PFC/3660545",
    "PNB/2730497",
    "POLYCAB/2455041",
    "RECLTD/3930881",
    "SAIL/758529",
    "SHRIRAMFIN/1102337",
    "TATACOMM/952577",
    "TRENT/502785",
    "TVSMOTOR/2170625",
    "UBL/4278529",
    "VOLTAS/951809",
    "ZEEL/975873",
    "ZYDUSLIFE/2029825"
]
# your full data here
dictionary = convert_to_dict(data)
a = {"Nifty50":dictionary}

In [17]:
stock = []
ticker = []
token = ['POLYPLEX/687873',
'FINPIPE/266497',
'PVRINOX/3365633',
'RCF/733697',
'GPPL/5051137',
'APLLTD/6483969',
'VAIBHAVGBL/2909185',
'KNRCON/3912449',
'CAMS/87553',
'MCX/7982337',
'EPL/251137',
'ENGINERSIN/1256193',
'CONCORDBIO/4623361',
'KARURVYSYA/470529',
'MAHLIFE/2060801',
'CUB/1459457',
'HEG/342017',
'GRINFRA/1293825',
'POWERINDIA/4724993',
'NAZARA/764673',
'REDINGTON/3649281',
'GMMPFAUDLR/401921',
'CENTRALBK/3812865',
'RADICO/2813441',
'CESC/160769',
'IBULHSGFIN/7712001',
'SYMPHONY/6192641',
'SHYAMMETL/1201409',
'LXCHEM/727297',
'FDC/1253889',
'MAPMYINDIA/1850113',
'RBA/382465',
'IDBI/377857',
'CIEINDIA/3823873',
'EQUITASBNK/233729',
'CREDITACC/1131777',
'WELSPUNLIV/2880769',
'HUDCO/5331201',
'INDIACEM/387841',
'PRAJIND/692481',
'EASEMYTRIP/714753',
'VGUARD/3932673',
'EXIDEIND/173057',
'MRPL/584449',
'GRAPHITE/151553',
'MGL/4488705',
'CHOLAHLDNG/5565441',
'CHAMBLFERT/163073',
'KAYNES/3095553',
'CEATLTD/3905025'


]
for i in range (0,len(token)):
    stock.append(token[i].split('/')[0])
    ticker.append(token[i].split('/')[1])

In [18]:
for i in range(0,len(ticker)):
    print(i,stock[i])
    tiks = ticker[i]
    stck = stock[i]
    #data = capital_market.price_volume_and_deliverable_position_data(symbol=tiks, from_date='22-02-2023', to_date='22-02-2024')
    df = createDffromDataS(fetchData(str(ticker[i])))
    #data.rename(columns = {'OpenPrice':'open','HighPrice':'high','LowPrice':'low','ClosePrice':'close','Date':'date'}, inplace = True) 
    #df = convert_specific_columns(data.copy())
    df = putAtrValues(df)
    
    dmzs = []
    for j in range(0,1):
        ltp = df['close'].iloc[-1]
        explosive1 = checkdata(df)
        explosive = explosive1.iloc[::-1].reset_index(drop=True)
        #print(explosive)
        obj = []
        for i in range(0,len(explosive['Date'])-1):
            base = explosive['index'][i]-1
            if explosive['Close'][i] > explosive['Open'][i] and abs(df['close'][base]-df['open'][base]) <= (explosive['High'][i]-explosive['Low'][i])*0.2 and explosive['Close'][i] > fiblvlprice(0.2,i) and df['open'][base] < fiblvlprice(0.5,i) and df['close'][base] < fiblvlprice(0.5,i) and explosive['Open'][i] < fiblvlprice(0.8,i) and df['high'][base] < fiblvlprice(0.5,i):
                obj.append(
                    {
                        "l":1,
                        "legoutDate":explosive['Date'][i],
                        "baseHigh":df['high'][base],
                        "baseLow":df['low'][base],
                        "dist":((df['high'][base]-ltp)/ltp)*100,
                        "ltp":ltp,
                        "chartData":{
                            "series":[
                                {
                                    "data":
                                    [
                                            {
                                    "x":df['date'][base],
                                    "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                                },
                                {
                                    "x":explosive['Date'][i],
                                    "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                                }
                                    ]
                                }
                            ],
                            "legoutDate":explosive['Date'][i]
                        }
                    })
                #epm and open[1] > open and close[1] < low and open[1] < fibprice(0.75) and high-low > high[1]-low[1]
            elif explosive['Close'][i] > explosive['Open'][i] and df['open'][base] > explosive['Open'][i] and df['close'][base] < explosive['Low'][i] and df['open'][base] < fiblvlprice(0.75,i) and explosive['High'][i]-explosive['Low'][i] > df['high'][base]-df['low'][base] and (explosive['Close'][i]-explosive['Close'][i])*0.25 > (df['open'][base]-df['close'][base]):
                obj.append(
                    {
                        "l":2,
                        "legoutDate":explosive['Date'][i],
                        "baseHigh":df['high'][base],
                        "baseLow":df['low'][base],
                        "dist":((df['high'][base]-ltp)/ltp)*100,
                        "ltp":ltp,
                        "chartData":{
                            "series":[
                                {
                                    "data":
                                    [
                                            {
                                    "x":df['date'][base],
                                    "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                                },
                                {
                                    "x":explosive['Date'][i],
                                    "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                                }
                                    ]
                                }
                            ],
                            "legoutDate":explosive['Date'][i]
                        }
                    })
                #close > open and epm and close[1] < fibprice(0.95) and open[1] > fibprice(1.25) and close[1] > open[1] and close > fibprice(0.2)
            elif explosive['Close'][i] > explosive['Open'][i] and df['close'][base] < fiblvlprice(0.95,i) and df['open'][base] > fiblvlprice(1.25,i) and df['close'][base] > df['open'][base] and explosive['Close'][i] > fiblvlprice(0.4,i):
                obj.append(
                    {
                        "l":3,
                        "legoutDate":explosive['Date'][i],
                        "baseHigh":df['high'][base],
                        "baseLow":df['low'][base],
                        "dist":((df['high'][base]-ltp)/ltp)*100,
                        "ltp":ltp,
                        "chartData":{
                            "series":[
                                {
                                    "data":
                                    [
                                            {
                                    "x":df['date'][base],
                                    "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                                },
                                {
                                    "x":explosive['Date'][i],
                                    "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                                }
                                    ]
                                }
                            ],
                            "legoutDate":explosive['Date'][i]
                        }
                    })
                #close > open and epm and close[1] < fibprice(0.95) and open[1] > fibprice(1.25) and close[1] > open[1] and close > fibprice(0.2)
            elif explosive['Close'][i] > explosive['Open'][i] and df['close'][base] < fiblvlprice(0.95,i) and df['open'][base] > fiblvlprice(1.3,i) and df['close'][base] > df['open'][base] and explosive['Close'][i] > fiblvlprice(0.2,i):
                obj.append(
                    {
                        "l":4,
                        "legoutDate":explosive['Date'][i],
                        "baseHigh":df['high'][base],
                        "baseLow":df['low'][base],
                        "dist":((df['high'][base]-ltp)/ltp)*100,
                        "ltp":ltp,
                        "chartData":{
                            "series":[
                                {
                                    "data":
                                    [
                                            {
                                    "x":df['date'][base],
                                    "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                                },
                                {
                                    "x":explosive['Date'][i],
                                    "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                                }
                                    ]
                                }
                            ],
                            "legoutDate":explosive['Date'][i]
                        }
                    })
            elif explosive['Close'][i] > explosive['Open'][i] and explosive['Close'][i] > fiblvlprice(0.8,i) and explosive['Open'][i] < fiblvlprice(0.2,i) and df['open'][base] > df['close'][base] and df['open'][base] < fiblvlprice(0.55,i) and df['close'][base] > fiblvlprice(0.9,i) and abs(df['close'][base]-df['open'][base]*5) < (explosive['Close'][i]-explosive['Open'][i]):
                obj.append(
                    {
                        "l":5,
                        "legoutDate":explosive['Date'][i],
                        "baseHigh":df['high'][base],
                        "baseLow":df['low'][base],
                        "dist":((df['high'][base]-ltp)/ltp)*100,
                        "ltp":ltp,
                        "chartData":{
                            "series":[
                                {
                                    "data":
                                    [
                                            {
                                    "x":df['date'][base],
                                    "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                                },
                                {
                                    "x":explosive['Date'][i],
                                    "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                                }
                                    ]
                                }
                            ],
                            "legoutDate":explosive['Date'][i]
                        }
                    })
            elif (explosive['Close'][i] > fiblvlprice(0.2,i) and explosive['Open'][i] < fiblvlprice(0.8,i)) and (df['close'][base] > fiblvlpricebase(0.2,base) and df['open'][base] < fiblvlpricebase(0.8,base)) and ((df['high'][base-1]-df['low'][base-1])<(df['high'][base]-df['low'][base])) and df['open'][base-1]<fiblvlpricebase(0.5,base) and df['close'][base-1]<fiblvlpricebase(0.5,base):
                obj.append(
                    {
                        "l":6,
                        "legoutDate":explosive['Date'][i],
                        "baseHigh":df['high'][base],
                        "baseLow":df['low'][base],
                        "dist":((df['high'][base]-ltp)/ltp)*100,
                        "ltp":ltp,
                        "chartData":{
                            "series":[
                                {
                                    "data":
                                    [
                                        {
                                            "x":df['date'][base-1],
                                            "y":[df['open'][base-1],df['high'][base-1],df['low'][base-1],df['close'][base-1]]
                                        },
                                            {
                                    "x":df['date'][base],
                                    "y":[df['open'][base],df['high'][base],df['low'][base],df['close'][base]]
                                },
                                {
                                    "x":explosive['Date'][i],
                                    "y":[explosive['Open'][i],explosive['High'][i],explosive['Low'][i],explosive['Close'][i]]
                                }
                                    ]
                                }
                            ],
                            "legoutDate":explosive['Date'][i]
                        }
                    })

        dmzs.append({tiks:obj})
        zones =[]
        for ti in range(0,1):
            brokenzone = []
            for i in range (0,len(dmzs[0][tiks])):

                for j in range(0,len(df['date'])):

                    if df['date'][j] == dmzs[0][tiks][i]['chartData']['legoutDate']:
                            #print(df['date'][j])
                        for k in range(j+1,len(df)):
                                #print(k,df['date'][k],df['low'][k],dmzs[0]['IOC'][i]['baseLow'])
                            if dmzs[0][tiks][i]['baseLow'] > df['low'][k]:
                                    #f0e0rti,dmzs[ti][tiks[ti]][i]['legoutDate'],'broken',dmzs[ti][tiks[ti]][i]['baseLow'],df['date'][k])
                                brokenzone.append(dmzs[0][tiks][i]['chartData']['legoutDate'])
                                break
        a = dmzs[0][tiks]
        bz_set = set(brokenzone)
            #print(tiks[ti])
            # Efficiently remove matching dictionaries from the end
        for i in range(len(a) - 1, -1, -1):
            if a[i]['legoutDate'] in bz_set:
                del a[i]
            #print(json.dumps(a,indent=4))
        zones.append({'ticker':stck,'ltp':ltp,'zone':a})
            
        for i in range(0,len(zones)):
            if len(zones[i]['zone']) > 0:
                print(mycollection.insert_one(zones[i]))

0 POLYPLEX
1 FINPIPE
2 PVRINOX
3 RCF
4 GPPL
5 APLLTD
6 VAIBHAVGBL
7 KNRCON
8 CAMS
9 MCX
10 EPL
11 ENGINERSIN
12 CONCORDBIO
13 KARURVYSYA
14 MAHLIFE
15 CUB
16 HEG
17 GRINFRA
18 POWERINDIA
19 NAZARA
20 REDINGTON
21 GMMPFAUDLR
22 CENTRALBK
23 RADICO
24 CESC
25 IBULHSGFIN
26 SYMPHONY
27 SHYAMMETL


C:\Users\Admin\AppData\Local\Temp\ipykernel_3944\3380140627.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if (bodyr/candr)>=ratio/100 and candr>=(rang*mp):


28 LXCHEM
29 FDC
30 MAPMYINDIA
31 RBA
32 IDBI
33 CIEINDIA
34 EQUITASBNK
35 CREDITACC
36 WELSPUNLIV
37 HUDCO


C:\Users\Admin\AppData\Local\Temp\ipykernel_3944\3380140627.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if (bodyr/candr)>=ratio/100 and candr>=(rang*mp):


38 INDIACEM
39 PRAJIND
40 EASEMYTRIP
41 VGUARD
42 EXIDEIND
43 MRPL


C:\Users\Admin\AppData\Local\Temp\ipykernel_3944\3380140627.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if (bodyr/candr)>=ratio/100 and candr>=(rang*mp):


44 GRAPHITE
45 MGL
46 CHOLAHLDNG
47 CHAMBLFERT
48 KAYNES
49 CEATLTD


In [18]:
midcap150 = ['POLICYBZR/1703937',
'LODHA/824321',
'SONACOMS/1199105',
'TORNTPOWER/3529217',
'DELHIVERY/2457345',
'NIACL/102145',
'INDIANB/3663105',
'VOLTAS/951809',
'CGPOWER/194561',
'AIAENG/3350017',
'TATACOMM/952577',
'SUPREMEIND/860929',
'JKCEMENT/3397121',
'GICRE/70913',
'FEDERALBNK/261889',
'UNOMINDA/3623425',
'MAXHEALTH/5728513',
'ABFRL/7707649',
'MFSL/548353',
'INDHOTEL/387073',
'KAJARIACER/462849',
'LAURUSLABS/4923905',
'OIL/4464129',
'GODREJPROP/4576001',
'BIOCON/2911489',
'LUPIN/2672641',
'PERSISTENT/4701441',
'MANKIND/3937281',
'JSL/2876417',
'ZEEL/975873',
'HDFCAMC/1086465',
'CUMMINSIND/486657',
'GODREJIND/2796801',
'SAIL/758529',
'BATAINDIA/94977',
'AUBANK/5436929',
'BDL/548865',
'ASTRAL/3691009',
'MPHASIS/1152769',
'OBEROIRLTY/5181953',
'HONAUT/874753',
'AUROPHARMA/70401',
'NAVINFLUOR/3756033',
'STARHEALTH/1813249',
'WHIRLPOOL/4610817',
'PEL/617473',
'POONAWALLA/2919169',
'SOLARINDS/3412993',
'RVNL/2445313',
'BHARATFORG/108033',
'NYKAA/1675521',
'SUNDARMFIN/854785',
'MAZDOCK/130305',
'RAMCOCEM/523009',
'FORTIS/3735553',
'PAYTM/1716481',
'GLAXO/295169',
'HINDZINC/364545',
'ASHOKLEY/54273',
'ENDURANCE/4818433',
'CRISIL/193793',
'RELAXO/6201601',
'GLAND/303617',
'BALKRISIND/85761',
'OFSS/2748929',
'COFORGE/2955009',
'EMAMILTD/3460353',
'IRFC/519425',
'YESBANK/3050241',
'JSWENERGY/4574465',
'SUMICHEM/4378881',
'AARTIIND/1793',
'POLYCAB/2455041',
'PFIZER/676609',
'UNIONBANK/2752769',
'TIINDIA/79873',
'FLUOROCHEM/3520001',
'TIMKEN/3634689',
'MSUMI/2200577',
'FACT/258049',
'M&MFIN/3400961',
'TATACHEM/871681',
'LINDEINDIA/416513',
'JUBLFOOD/4632577',
'KPITTECH/2478849',
'SCHAEFFLER/258817',
'BANDHANBNK/579329',
'ATUL/67329',
'ADANIPOWER-BE/4452353',
'LTTS/4752385',
'AJANTPHARM/2079745',
'TATAELXSI/873217',
'THERMAX/889601',
'IDFCFIRSTB/2863105',
'DEEPAKNTR/5105409',
'MRF/582913',
'ABCAPITAL/5533185',
'SUNDRMFAST/856321',
'PATANJALI/4359425',
'TRIDENT/2479361',
'ZFCVINDIA/4330241',
'MAHABANK/2912513',
'UBL/4278529',
'PHOENIXLTD/3725313'
'PRESTIGE/5197313',
'L&TFH/6386689',
'KANSAINER/306177',
'ESCORTS/245249',
'PAGEIND/3689729',
'LICHSGFIN/511233',
'ISEC/637185',
'GUJGASLTD/2713345',
'IPCALAB/418049',
'CONCOR/1215745',
'BAYERCROP/4589313',
'NMDC/3924993',
'APOLLOTYRE/41729',
'LALPATHLAB/2983425',
'PETRONET/2905857',
'RAJESHEXPO/1894657',
'COROMANDEL/189185',
'NHPC/4454401',
'BHEL/112129',
'SUNTV/3431425',
'DALBHARAT/2067201',
'IGL/2883073',
'GMRINFRA/3463169',
'KPRMILL/3817473',
'HINDPETRO/359937',
'METROBRAND/1853953',
'PFC/3660545',
'INDUSTOWER/7458561',
'CARBORUNIV/152321',
'DIXON/5552641',
'DEVYANI/1375489',
'RECLTD/3930881',
'IDEA/3677697'
]

In [16]:
sm250 = [
'JAMNAAUTO/5319169',
'AVANTIFEED/2031617',
'SUVENPHAR/4593921',
'BORORENEW/806145',
'WELCORP/3026177',
'METROPOLIS/2452737',
'IIFL/3023105',
'ELGIEQUIP/239873',
'VARROC/987393',
'CHEMPLASTS/1394945',
'HOMEFIRST/526337',
'KSB/498945',
'RBLBANK/4708097',
'RATNAMANI/3443457',
'NCC/593665',
'SWANENERGY/6936321',
'SYRMA/2763009',
'JKLAKSHMI/3453697',
'JINDALSAW/774145',
'BLUESTARCO/2127617',
'CENTURYTEX/160001',
'JMFINANCIL/3491073',
'HFCL/5619457',
'PCBL/678145',
'LAXMIMACH/506625',
'LEMONTREE/667137',
'DELTACORP/3851265',
'SFL/4911105',
'SPARC-BE/3786753',
'TATAINVEST-BE/1698817',
'SOBHA/3539457',
'BEML/101121',
'BLS/4423425',
'AARTIDRUGS/1147137',
'CASTROLIND/320001',
'STLTECH/2383105',
'ANGELONE/82945',
'CGCL/5204225',
'ANGELONE/82945',
'CGCL/5204225',
'NATIONALUM/1629185',
'GUJALKALI/324353',
'CRAFTSMAN/730625',
'CANFINHOME/149249',
'MANAPPURAM/4879617',
'GPIL/3432705',
'UCOBANK/2873089',
'GAEL/2259969',
'SAREGAMA/1252353',
'PNBHOUSING/4840449',
'RHIM/7977729',
'IRB/3920129',
'POLYMED/6583809',
'LATENTVIEW/1745409',
'IEX/56321',
'SUPRAJIT/2992385',
'NUVOCO/1389057',
'RALLIS/720897',
'MMTC/4596993',
'MOTILALOFS/3826433',
'360ONE/3343617',
'GSFC/319233',
'POLYPLEX/687873',
'FINPIPE/266497',
'PVRINOX/3365633',
'RCF/733697',
'GPPL/5051137',
'APLLTD/6483969',
'VAIBHAVGBL/2909185',
'KNRCON/3912449',
'CAMS/87553',
'MCX/7982337',
'EPL/251137',
'ENGINERSIN/1256193',
'CONCORDBIO/4623361',
'KARURVYSYA/470529',
'MAHLIFE/2060801',
'CUB/1459457',
'HEG/342017',
'GRINFRA/1293825',
'POWERINDIA/4724993',
'NAZARA/764673',
'REDINGTON/3649281',
'GMMPFAUDLR/401921',
'CENTRALBK/3812865',
'RADICO/2813441',
'CESC/160769',
'IBULHSGFIN/7712001',
'SYMPHONY/6192641',
'SHYAMMETL/1201409',
'LXCHEM/727297',
'FDC/1253889',
'MAPMYINDIA/1850113',
'RBA/382465',
'IDBI/377857',
'CIEINDIA/3823873',
'EQUITASBNK/233729',
'CREDITACC/1131777',
'WELSPUNLIV/2880769',
'HUDCO/5331201',
'INDIACEM/387841',
'PRAJIND/692481',
'EASEMYTRIP/714753',
'VGUARD/3932673',
'EXIDEIND/173057',
'MRPL/584449',
'GRAPHITE/151553',
'MGL/4488705',
'CHOLAHLDNG/5565441',
'CHAMBLFERT/163073',
'KAYNES/3095553',
'CEATLTD/3905025'

]